In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d mohamedchahed/dog-breeds

100% 82.0M/82.3M [00:06<00:00, 19.4MB/s]
100% 82.3M/82.3M [00:06<00:00, 13.5MB/s]


In [4]:
!unzip -q dog-breeds.zip

In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Путь к папке с изображениями
data_dir = 'dog-breeds'

# Путь к папке, куда сохранятся данные для тренировки
train_dir = 'train'

# Путь к папке, куда сохранятся данные для тестирования
test_dir = 'test'

# Создаем папки для тренировочных и тестовых данных
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Список названий подпапок
class_names = os.listdir(data_dir)

# Для каждой подпапки создаем папки внутри train_dir и test_dir
for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

# Для каждой подпапки выбираем тренировочные и тестовые данные
for class_name in class_names:
    class_dir = os.path.join(data_dir, class_name)
    images = os.listdir(class_dir)
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    
    # Копируем изображения в соответствующие папки
    for train_image in train_images:
        src = os.path.join(class_dir, train_image)
        dst = os.path.join(train_dir, class_name, train_image)
        shutil.copy(src, dst)
    
    for test_image in test_images:
        src = os.path.join(class_dir, test_image)
        dst = os.path.join(test_dir, class_name, test_image)
        shutil.copy(src, dst)

In [6]:
from keras.preprocessing.image import ImageDataGenerator

# Определяем параметры для аугментации
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # Добавляем валидационную выборку
)

test_datagen = ImageDataGenerator(rescale=1./255)


# Создаем генераторы для тренировочных и тестовых данных
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Используем только тренировочные данные
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation' # Используем только валидационные данные
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 344 images belonging to 8 classes.
Found 84 images belonging to 8 classes.
Found 113 images belonging to 8 classes.


In [32]:
from keras.models import Model
from keras.layers import Input, Dropout, GlobalAveragePooling2D, Flatten, Dense, BatchNormalization

input_shape = (224, 224, 3)

base_model = tf.keras.applications.ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
base_model.trainable = False

x = base_model.output
x = Dropout(0.25)(x)
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(8, activation='softmax')(x)

res_model = Model(inputs=base_model.input, outputs=output)

res_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [38]:
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Компиляция модели
opt = Adam(learning_rate=1e-4)
res_model.compile(loss="categorical_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy"])

# Определение callback функций
my_callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, verbose=1, mode="auto"),
    ModelCheckpoint(
        filepath="./models/best_resnetmodel_DogBreeds.h5",
        monitor="val_loss",
        verbose=1,
        save_best_only=True,
        mode="auto"
    ),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1, mode='auto')
]

In [39]:
Batch_size = 8
EPOCHS = 25

In [40]:
history = res_model.fit(train_generator, 
                        batch_size=Batch_size,
                        epochs=EPOCHS,
                        callbacks=my_callbacks,
                        validation_data=validation_generator)

Epoch 1/25
11/11 [==============================] - ETA: 0s - loss: 0.1108 - accuracy: 0.9709
Epoch 1: val_loss improved from inf to 0.28666, saving model to ./models/best_resnetmodel_DogBreeds.h5
11/11 [==============================] - 17s 1s/step - loss: 0.1108 - accuracy: 0.9709 - val_loss: 0.2867 - val_accuracy: 0.9167 - lr: 1.0000e-04
Epoch 2/25
11/11 [==============================] - ETA: 0s - loss: 0.0905 - accuracy: 0.9884
Epoch 2: val_loss improved from 0.28666 to 0.26882, saving model to ./models/best_resnetmodel_DogBreeds.h5
11/11 [==============================] - 13s 1s/step - loss: 0.0905 - accuracy: 0.9884 - val_loss: 0.2688 - val_accuracy: 0.9167 - lr: 1.0000e-04
Epoch 3/25
11/11 [==============================] - ETA: 0s - loss: 0.0504 - accuracy: 0.9884
Epoch 3: val_loss improved from 0.26882 to 0.25917, saving model to ./models/best_resnetmodel_DogBreeds.h5
11/11 [==============================] - 13s 1s/step - loss: 0.0504 - accuracy: 0.9884 - val_loss: 0.2592 - v

In [41]:
from keras.models import load_model

model = load_model('models/best_resnetmodel_DogBreeds.h5')

In [42]:
model.evaluate(test_generator)

4/4 [==============================] - 3s 396ms/step - loss: 0.0925 - accuracy: 0.9646


[0.09253579378128052, 0.9646017551422119]